In [1]:
# First, check if the DB file for this notebook exists and remove it. 
import os

if os.path.exists('example.db'):
    os.remove('example.db')

In [2]:
import sqlite3
from sqlalchemy import create_engine

engine = create_engine('sqlite:///example.db', echo=True)

For example, use a "Declarative Base" object to use an "object-model" approach.  We can defined Tables by themselves using SQLAlchemy, but we're not going to do that right now.  

In [3]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

This uses the Declarative style to create a "records" object with the fields we want.  

Please note that in the following definition, by keeping only one "detector" and one "enabled" field, we remove the ability for unique indexing on the SeriesNumber and EventNumber fields.

In [4]:
from sqlalchemy import Column, Integer, BigInteger, String, DateTime, Float, Boolean
from sqlalchemy import Sequence

class RecordModel(Base):
    __tablename__ = 'records'
    
    recordID = Column(Integer, 
                      Sequence('record_id_seq'), 
                      primary_key=True)
    SeriesNumber = Column(BigInteger, 
                          index=True, 
                          #unique=True, 
                          nullable=False)
    EventNumber = Column(BigInteger,
                         index=True,
                         #unique=True,
                         nullable=False)
    DetNum = Column(Integer)
    DetType = Column(Integer) 
    Enabled = Column(Boolean)
    
    
    def __repr__(self):
        return "<Record(recordID='%s', SeriesNumber='%s', EventNumber=''%s')" % (
                    self.recordID, self.SeriesNumber, self.EventNumber)
    
        

Using the declarative style automatically creates the indexed "metadata schema" for the database, which will speed up queries and make the database easily sharable.  

In [5]:
RecordModel.__table__

Table('records', MetaData(bind=None), Column('recordID', Integer(), table=<records>, primary_key=True, nullable=False, default=Sequence('record_id_seq', metadata=MetaData(bind=None))), Column('SeriesNumber', BigInteger(), table=<records>, nullable=False), Column('EventNumber', BigInteger(), table=<records>, nullable=False), Column('DetNum', Integer(), table=<records>), Column('DetType', Integer(), table=<records>), Column('Enabled', Boolean(), table=<records>), schema=None)

Now let's actually create the SQLite database:

In [6]:
Base.metadata.create_all(engine)

2020-06-12 12:46:18,592 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-06-12 12:46:18,594 INFO sqlalchemy.engine.base.Engine ()
2020-06-12 12:46:18,598 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-06-12 12:46:18,599 INFO sqlalchemy.engine.base.Engine ()
2020-06-12 12:46:18,602 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("records")
2020-06-12 12:46:18,603 INFO sqlalchemy.engine.base.Engine ()
2020-06-12 12:46:18,605 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("records")
2020-06-12 12:46:18,607 INFO sqlalchemy.engine.base.Engine ()
2020-06-12 12:46:18,611 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE records (
	"recordID" INTEGER NOT NULL, 
	"SeriesNumber" BIGINT NOT NULL, 
	"EventNumber" BIGINT NOT NULL, 
	"DetNum" INTEGER, 
	"DetType" INTEGER, 
	"Enabled" BOOLEAN, 
	PRIMARY KEY ("recordID"), 
	CHECK ("Enabled" IN (0, 1))
)


2020-06-12 12:46:18,612 

Let's create some example entries, and insert them in the database.

In [7]:
from sqlalchemy.orm import sessionmaker

# Use a 'factory' for creating a connection to our database.
Session = sessionmaker(bind=engine)

# Create actual session object we will use to interact with DB
session = Session()

In [8]:
%%time
# Create some example records
import random
import numpy as np

before_commit = []

for i in range(0,8500):   
    for j in range(0, 35):
        _enable = random.choice([True, False])
        new_record = RecordModel(SeriesNumber=1005,
                                 EventNumber=500+i,
                                 DetNum=j,
                                 DetType=int(j%10),
                                 Enabled=_enable)
        before_commit.append(new_record)
    


CPU times: user 8.71 s, sys: 203 ms, total: 8.92 s
Wall time: 8.97 s


In [9]:
# how many objects?

print(len(before_commit))

297500


In [10]:
%%time
# Add them to the session and save session to DB, aka "Database Transaction"

session.bulk_save_objects(before_commit)
session.commit()

2020-06-12 12:46:28,273 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-06-12 12:46:30,824 INFO sqlalchemy.engine.base.Engine INSERT INTO records ("SeriesNumber", "EventNumber", "DetNum", "DetType", "Enabled") VALUES (?, ?, ?, ?, ?)
2020-06-12 12:46:30,824 INFO sqlalchemy.engine.base.Engine ((1005, 500, 0, 0, 1), (1005, 500, 1, 1, 0), (1005, 500, 2, 2, 1), (1005, 500, 3, 3, 0), (1005, 500, 4, 4, 0), (1005, 500, 5, 5, 0), (1005, 500, 6, 6, 1), (1005, 500, 7, 7, 1)  ... displaying 10 of 297500 total bound parameter sets ...  (1005, 8999, 33, 3, 1), (1005, 8999, 34, 4, 0))
2020-06-12 12:46:31,838 INFO sqlalchemy.engine.base.Engine COMMIT
CPU times: user 3.96 s, sys: 160 ms, total: 4.12 s
Wall time: 4.21 s


What's our resulting filesize? (n.b. Size is in "MebiBytes" [MiB]).

In [11]:
import os

bytecount = os.stat('example.db').st_size
print("DB Size is ", (bytecount * (9.537E-7)), " [MiB]")

DB Size is  12.437834956800002  [MiB]


In [21]:
# Now I want to form the Pass/Cut True/False array for one particular series+event

target_series = 1005
target_event = (4700)+500

eval_query = session.query(RecordModel).order_by(RecordModel.DetNum).\
                filter(RecordModel.SeriesNumber == target_series).\
                filter(RecordModel.EventNumber == target_event)

pass_cuts = np.array([record.Enabled for record in eval_query], dtype=bool)
    
pass_cuts

2020-06-12 13:01:21,388 INFO sqlalchemy.engine.base.Engine SELECT records."recordID" AS "records_recordID", records."SeriesNumber" AS "records_SeriesNumber", records."EventNumber" AS "records_EventNumber", records."DetNum" AS "records_DetNum", records."DetType" AS "records_DetType", records."Enabled" AS "records_Enabled" 
FROM records 
WHERE records."SeriesNumber" = ? AND records."EventNumber" = ? ORDER BY records."DetNum"
2020-06-12 13:01:21,390 INFO sqlalchemy.engine.base.Engine (1005, 5200)


array([False,  True, False, False,  True, False,  True, False, False,
       False,  True, False,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False,  True, False, False, False,  True, False])

In [23]:
cuts_dettype = np.array([record.DetType for record in eval_query], dtype=np.int32)

cuts_dettype

2020-06-12 13:02:03,076 INFO sqlalchemy.engine.base.Engine SELECT records."recordID" AS "records_recordID", records."SeriesNumber" AS "records_SeriesNumber", records."EventNumber" AS "records_EventNumber", records."DetNum" AS "records_DetNum", records."DetType" AS "records_DetType", records."Enabled" AS "records_Enabled" 
FROM records 
WHERE records."SeriesNumber" = ? AND records."EventNumber" = ? ORDER BY records."DetNum"
2020-06-12 13:02:03,078 INFO sqlalchemy.engine.base.Engine (1005, 5200)


array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1,
       2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4], dtype=int32)

## do it from scratch

In [4]:
## Set up the basics
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

new_connection = create_engine('sqlite:///example.db', echo=True)

# Use a 'factory' for creating a connection to our database.
Session = sessionmaker(bind=new_connection)

# Create actual session object we will use to interact with DB
new_session = Session()

## Automap schema from the database itself
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

Base = automap_base()
Base.prepare(new_connection, reflect=True)

RecordModel = Base.classes.records

## And query
import numpy as np

target_series = 1005
target_event = (4700)+500

eval_query = new_session.query(RecordModel).order_by(RecordModel.DetNum).\
                filter(RecordModel.SeriesNumber == target_series).\
                filter(RecordModel.EventNumber == target_event)

pass_cuts = np.array([record.Enabled for record in eval_query], dtype=bool)
    
pass_cuts

2020-06-12 14:50:28,560 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-06-12 14:50:28,561 INFO sqlalchemy.engine.base.Engine ()
2020-06-12 14:50:28,563 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-06-12 14:50:28,564 INFO sqlalchemy.engine.base.Engine ()
2020-06-12 14:50:28,566 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-06-12 14:50:28,567 INFO sqlalchemy.engine.base.Engine ()
2020-06-12 14:50:28,570 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("records")
2020-06-12 14:50:28,571 INFO sqlalchemy.engine.base.Engine ()
2020-06-12 14:50:28,575 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'records' AND type = 'table'
2020-06-12 14:50:28,576 INFO sqlalchemy.engine.base.Engine ()
2020-06-12 14:50:28,579 INFO sqlalchemy.eng

array([False,  True, False, False,  True, False,  True, False, False,
       False,  True, False,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False,  True, False, False, False,  True, False])